Parameter Setting

In [ ]:
dropoutRatio = 0.0
dataDir = '/gdrive/My Drive/tiny-imagenet-200'              # directory of Tiny ImageNet
modelDir = "models/"                                        # directory for saving trined model
model_name = "TinyImageNetdropoutRatio" + str(dropoutRatio) # model name 

Connect to gdirve for collaboratory user

In [ ]:
# connect to gdrive

#from google.colab import drive
#drive.mount('/gdrive')
#%cd /gdrive/My\ Drive/workDir

Set training and validation data

In [ ]:
import keras, os
import numpy as np
import glob
from keras.models import Sequential
from keras.preprocessing.image import load_img, img_to_array, array_to_img

In [ ]:
train_images = []
train_labels = []
valid_images = []
valid_labels = []

In [ ]:
if os.path.isfile( 'train_images.npy' ) and os.path.isfile( 'train_labels.npy' ) \
  and os.path.isfile( 'valid_images.npy' ) and os.path.isfile( 'valid_labels.npy' ):
  train_images = np.load("train_images.npy")
  train_labels = np.load("train_labels.npy")
  valid_images = np.load("valid_images.npy")
  valid_labels = np.load("valid_labels.npy")
else:
  dirList = os.listdir( dataDir + "/train" )
  calNum = 0
  for d in dirList:
    #set train images
    files = glob.glob( dataDir + "/train/" + d + "/images/" + "*.JPEG" )
    for imgfile in files:
      img = load_img( imgfile, target_size = (64, 64))
      array = img_to_array(img) / 255 - 0.5
      train_images.append(array)
      train_labels.append(calNum)
    #set valid images
    with open( dataDir + "/val/"+ 'val_annotations.txt') as f:
      while True:
        line = f.readline()
        if line:
          splitLine = line.split()
          if( splitLine[1] == d ):
            img = load_img( dataDir + "/val/images/" + splitLine[0] )
            array = img_to_array( img ) / 255 - 0.5
            valid_images.append(array)
            valid_lables.append(calNum)
        else:
          break
    calNum += 1
  #save the training images
  nplist = np.array(train_images)
  np.save("train_images2", nplist)
  nplist = np.array(train_labels)
  np.save("train_labels2", nplist)
  nplist = np.array(valid_images)
  np.save("valid_images2", nplist)
  nplist = np.array(valid_labels)
  np.save("valid_labels2", nplist)


In [ ]:
from keras.utils.np_utils import to_categorical
train_labels = to_categorical(train_labels)
valid_labels = to_categorical(valid_labels)

Set Model

In [ ]:
from keras import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPool2D

In [ ]:
from keras.applications.vgg16 import VGG16
base_model = VGG16(input_shape = (64,64,3), include_top=False, weights = 'imagenet')

In [ ]:
# freeze layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
import keras
model = Sequential([
    base_model,
    keras.layers.Flatten(),
    keras.layers.Dropout(dropoutRatio),
    keras.layers.Dense(1000, activation="relu"),
    keras.layers.Dropout(dropoutRatio),
    keras.layers.Dense(200, activation="softmax")
])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1000)              2049000   
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 200)               200200    
Total params: 16,963,888
Trainable params: 2,249,200
Non-trainable params: 14,714,688
__________________________________

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
fit = model.fit(train_images, train_labels,
              epochs=30, batch_size=128,
              validation_data=(valid_images, valid_labels),
              shuffle=True)

Epoch 1/30
782/782 [==============================] - 51s 29ms/step - loss: 4.4323 - accuracy: 0.0967 - val_loss: 3.3673 - val_accuracy: 0.2406
Epoch 2/30
782/782 [==============================] - 21s 27ms/step - loss: 3.5192 - accuracy: 0.2159 - val_loss: 3.2263 - val_accuracy: 0.2734
Epoch 3/30
782/782 [==============================] - 21s 27ms/step - loss: 3.4359 - accuracy: 0.2309 - val_loss: 3.2355 - val_accuracy: 0.2773
Epoch 4/30
782/782 [==============================] - 21s 27ms/step - loss: 3.4158 - accuracy: 0.2407 - val_loss: 3.2126 - val_accuracy: 0.2815
Epoch 5/30
782/782 [==============================] - 21s 27ms/step - loss: 3.4358 - accuracy: 0.2427 - val_loss: 3.2868 - val_accuracy: 0.2732
Epoch 6/30
782/782 [==============================] - 21s 27ms/step - loss: 3.4822 - accuracy: 0.2395 - val_loss: 3.3271 - val_accuracy: 0.2670
Epoch 7/30
782/782 [==============================] - 21s 27ms/step - loss: 3.5149 - accuracy: 0.2394 - val_loss: 3.2937 - val_accuracy:

Save Model

In [ ]:
model.save( modelDir + model_name+".h5", include_optimizer = False)